# PyOpenCL Parallel Patterns: Map/Elementwise

## Setup code

In [6]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import numpy as np

In [16]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

In [17]:
n = 10**7
a = cl.clrandom.rand(queue, n, np.float32)
b = cl.clrandom.rand(queue, n, np.float32)

## A simple 'target application'

We would like to evaluate this linear combination:

In [18]:
c1 = 5*a + 6*b

A problem with this is that every single operator (all three of them--and easily more for complicated expressions) corresponds to a kernel call, which can lead to high overhead. Let's try and avoid that by stuffing the entire operation into one kernel, in turn saving lots of memory traffic:

In [19]:
from pyopencl.elementwise import ElementwiseKernel

Arguments: `ElementwiseKernel(ctx, c_args, operation)`

In [21]:
c2 = cl.array.empty_like(a)
lin_comb(5, a, 6, b, c2)

In [28]:
import numpy.linalg as la
print(la.norm(c1.get() - c2.get(), ord=np.inf))

## Timing ElementwiseKernel

Did this optimization pay off?

In [29]:
from time import time
queue.finish()
start_time = time()

for i in range(10):
    c1 = 5*a + 6*b
    
queue.finish()
print("elapsed: {0} s".format(time()-start_time))

In [30]:
from time import time
queue.finish()
start_time = time()

for i in range(10):
    lin_comb(5, a, 6, b, c2)
    
queue.finish()
print("elapsed: {0} s".format(time()-start_time))